Cloverleaf / Geometric Method

### Preamble

Version Control

v1.0 - initial release

In [ ]:
# imports

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')

In [ ]:
# constants and conversion factors

m2ft = 3.28084
ft2m = 1 / m2ft
kt2ms = 1852 / 60 / 60 # 1 nm in meters; from hours to seconds
ms2kt = 1 / kt2ms
C2K = 273.15
d2r = np.pi / 180
r2d = 1 / d2r

In [ ]:
# International Standard Atmosphere

T0 = 15 + C2K # (K)
p0 = 101325 # (Pa)
L = -6.5 / 1000 # (K/m)
a0 = 340.3 # (m/s)
rho0 = 1.225 # (kg/m3)
R =  p0 / (rho0 * T0) # air, specific, std ISA in (J/kg)/K
g_zero = 9.80665 # (m/s2)
gamma = 1.4 # adiabatic index for air
Cp = 1006 # (J/kg)/K

In [ ]:
# data from airplane / instruments calibration

#  speeds
Vmo = 350 # (kts)
Vsr0 = 105 # (kts)

#  instrument errors
delta_Vic = 2 # (kts)
delta_Hic = -20 # (ft)
delta_Tic = -1 # (C)
Kt = 1.0 # temperature recovery factor

In [ ]:
# FT data

# from airplane anemometry
Vi = np.array([117, 116, 118]) # (KIAS)
Hi = np.array([6000, 6000, 6000]) # (ft)
Ti = np.array([11, 11, 11]) # (C)

# from airplane GPS
Vg = np.array([138, 133, 120]) # (Kts)
sigma_g = np.array([7, 114, 234]) # (degrees)

## Anemometric Side

### Correct for Instrument Errors

$V_{ic}=V{i}+\Delta V_{ic}$

$H_{ic}=H_{Pi}+\Delta H_{ic}$

$T_{ic}=T_{i}+\Delta T_{ic}$

In [ ]:
Vic = Vi + delta_Vic # (kts)
Hic = Hi + delta_Hic # (ft)
Tic = Ti + delta_Tic + C2K # (K)
print(f'Vic = {Vic} KIAS')
print(f'Hic = {Hic} ft')
print(f'Tic = {Tic} K')

Find the $\frac{q_{cic}}{P_0}$ ratio, which will be used to find the Mach number and static port position error later:

$\frac{q_{cic}}{P_0}=(1+0.2(\frac{V_{ic}}{a_0})^2)^{\frac{7}{2}}-1$ 

In [ ]:
qcic_over_p0 = (1 + 0.2 * ((Vic * kt2ms) / a0)**2)**(7 / 2) - 1
print(f'qcic_over_p0: {qcic_over_p0}')

Calculate the pressure ratio $\delta_{ic}$

$\delta_{ic}=(1 + \frac{L}{T_0}*H_{ic}^{(\frac{-g_0}{R * L})})$

In [ ]:
deltaISA_ic = (1 + L / T0 * (Hic * ft2m))**(-g_zero / (R * L))
print(f'deltaISA_ic: {deltaISA_ic}')

Find:

$\frac{q_{cic}}{p_s}=\frac{q_{cic}}{p_0}*\frac{p_0}{p_s}=\frac{q_{cic}}{p_0}*\frac{1}{\delta_{ic}}$ 

In [ ]:
qcic_over_ps = qcic_over_p0 / deltaISA_ic
print(f'qcic_over_ps: {qcic_over_ps}')

and the indicated Mach number from:

$M_{ic} = \sqrt{5*[(\frac{q_{cic}}{p_s}+1)^{\frac{2}{7}}-1]}$

In [ ]:
Mic = np.sqrt(5 * ((qcic_over_ps + 1)**(2/7) - 1))
print(f'Indicated Mach number: {Mic}')

Then, from indicated $M_{ic}$, total over static port pressure ratio ->   $\frac{p_T}{p_s}=(1+0.2 M_{ic}^2)^\frac{7}{2}$

In [ ]:
pt_over_ps = (1 + 0.2 * Mic.mean()**2)**(7 / 2)
print(f'pt_over_ps: {pt_over_ps}')

## Cloverleaf / Geometric Method

![038](pictures/wind_triangles.png)

## GPS Side

First step is to find the East and North components of the ground speed:

$V_{g_E}=V{g}*sin( \sigma_g)$ and

$V_{g_N}=V{g}*cos( \sigma_g)$

In [ ]:
# ground speed components

VgE = Vg * np.sin(sigma_g * d2r)
VgN = Vg * np.cos(sigma_g * d2r)

![042](pictures/GPS_geometric_method_14.png)

## Bisectors

line between $V_{g_{1}}$ and $V_{g_{2}}$

The midpoint is easy:

$(X_{mid_{1,2}}, Y_{mid_{1,2}}) = (\frac{V_{gx_{1}}+V_{gx_{2}}}{2},\frac{V_{gy_{1}}+V_{gy_{2}}}{2})$

In [ ]:
# midpoints

# runs 1-2
X_mid12 = (VgE[0] + VgE[1]) / 2
Y_mid12 = (VgN[0] + VgN[1]) / 2
# runs 1-3
X_mid13 = (VgE[0] + VgE[2]) / 2
Y_mid13 = (VgN[0] + VgN[2]) / 2
# runs 2-3
X_mid23 = (VgE[1] + VgE[2]) / 2
Y_mid23 = (VgN[1] + VgN[2]) / 2

The slope is rise over run: 

$Slope_{1,2}=\frac{V_{gN_{2}}-V_{gN_{1}}}{V_{gE_{2}}-V_{gE_{1}}}$

In [ ]:
# slopes

# runs 1-2
m12 = (VgN[1] - VgN[0]) / (VgE[1] - VgE[0])
# runs 1-3
m13 = (VgN[2] - VgN[0]) / (VgE[2] - VgE[0])
# runs 2-3
m23 = (VgN[1] - VgN[2]) / (VgE[1] - VgE[2])

To get the slope of the perpendicular, we get its negative inverse:

$Slope_{1,2_{\perp}}=-\frac{1}{Slope_{1,2}}$

In [ ]:
# perpendicular slopes

# runs 1-2
m12_p = -1 / m12
# runs 1-3
m13_p = -1 / m13
# runs 2-3
m23_p = -1 / m23

From the equation for a line: $y=mx+b$, we can isolate the intercept to get $b=y-mx$

Let's call $Slope_{1,2_{\perp}}=m_{1,2p}$;

$b_{1,2}=Y_{mid_{1,2}}-m_{1,2p}*(X_{mid_{1,2}})$

In [ ]:
# line intercepts

# runs 1-2
b12 = Y_mid12 - m12_p * X_mid12
# runs 1-3
b13 = Y_mid13 - m13_p * X_mid13
# runs 2-3
b23 = Y_mid23 - m23_p * X_mid23

We can use the equations for the 2 bisectors and get their intersection:


$Y_{intcpt}=m_{1,2p} X_{intcpt} + b_{1,2}$ for the bisector line between $V_{g_{1}}$ and $V_{g_{2}}$

$Y_{intcpt}=m_{1,3p} X_{intcpt} + b_{1,3}$ for the bisector line between $V_{g_{1}}$ and $V_{g_{3}}$

From which we can equate both and write:

$Y_{intcpt}=m_{1,2p}X_{intcpt} + b_{1,2}=m_{1,3p}X_{intcpt} + b_{1,3}$ 

to solve for $X_{intcpt}$

$X_{intcpt}=\frac{b_{1,2}-b_{1,3}}{m_{1,3p}-m_{1,2p}}$

In [ ]:
# X direction (East) wind component

# runs 1-3
VwE13 = (b12 - b13) / (m13_p - m12_p)
print(f'Calculated East Wind Component runs 1-3: {VwE13} kts')
# runs 2-3
VwE23 = (b23 - b13) / (m13_p - m23_p)
print(f'Calculated East Wind Component runs 2-3: {VwE23} kts')
# runs 1-2
VwE12 = (b23 - b12) / (m12_p - m23_p)
print(f'Calculated East Wind Component runs 1-2: {VwE12} kts')

Once we have the value for $X_{intcpt}$, which is $V_{wE}$, we plug it back into $Y_{intcpt}=m_{1,2p} X_{intcpt} + b_{1,2}$ to get $V_{wN}$

In [ ]:
# Y direction wind component

# runs 1-3
VwN13 = m12_p * VwE13 + b12
print(f'Calculated North Wind Component runs 1-3: {VwN13} kts')
# runs 2-3
VwN23 = m23_p * VwE23 + b23
print(f'Calculated North Wind Component runs 2-3: {VwN23} kts')
# runs 1-2
VwN12 = m12_p * VwE12 + b12
print(f'Calculated North Wind Component runs 1-2: {VwN12} kts')

Now we can average out the components:

In [ ]:
VwE = (VwE13 + VwE23 + VwE12) / 3
VwN = (VwN13 + VwN23 + VwN12) / 3

The wind speed is then $V_w=\sqrt{V_{wE}^2 + V_{wN}^2}$...

In [ ]:
# wind speed (Pythagoras)

Vw = np.sqrt(VwE**2 + VwN**2)
print(f'Calculated Wind Speed  : {Vw} kts')

Wind direction:

Note that if we apply the arctangent directly, it will result in the wrong quadrant. So we need to correct for this by:

$\psi_w = (arctan(V_{wE},V_{wN})) + 2\pi) \bmod (2 \pi)$

In [ ]:
# wind direction

psi_w = (np.arctan2(VwE, VwN) + 2 * np.pi) % (2 * np.pi) # (rad)
print(f'Calculated Wind Direction  : {psi_w*r2d} degrees')

The true airspeed magnitude is obtained from, (remember, assuming $V_{ti}$ it was constant for the 3 runs, we can use any of the runs $V_{g}$):

$V_t=\sqrt{(V_{gE_{1}}-V_{wE})^2+(V_{gN_{1}}-V_{wN})^2}$

In [ ]:
# true airspeed

Vt = np.sqrt((VgE[0] - VwE)**2 + (VgN[0] - VwN)**2)
print(f'Truth True Airspeed  : {Vt} kts')

### True Mach

Start with temperature...

$T_a = \bar{T_{ic}} - \frac{K_t V_{t}^2}{2C_p}$

In [ ]:
Ta = Tic.mean() - (Kt * (Vt * kt2ms)**2) / (2 * Cp)
print(f'Ambient temperature: {Ta} K')

Recall that $\theta_{test}=\frac{T_{a}}{T_0}$

In [ ]:
theta_test = Ta / T0

Truth Mach number is then

$M=\frac{V_t}{a_0*\sqrt{\theta_{test}}}$

In [ ]:
M = (Vt * kt2ms) / (a0 * np.sqrt(theta_test))
print(f'truth Mach: {M}')

Calculate total over ambient pressure ratio->   $\frac{p_T}{p_a}=(1+0.2M^2)^\frac{7}{2}$

In [ ]:
pt_over_pa = (1 + 0.2 * M**2)**(7 / 2)
print(f'pt_over_pa: {pt_over_pa}')

Finally, the position error ratio is:

$\frac{\Delta p_s}{p_s}=(\frac{1}{\frac{p_T}{p_s}}-\frac{1}{\frac{p_T}{p_a}})\frac{p_T}{p_s}$

In [ ]:
delta_ps_over_ps = ((1 / pt_over_ps) - (1 / pt_over_pa)) * pt_over_ps
print(f'delta_ps_over_ps: {delta_ps_over_ps}')

_____ common part _____

In [ ]:
delta_ps_over_ps_runs = np.array([-0.0004657, 0.0012954, 0.0024276, -0.0006998, 0.0019471, 0.0026558])
Vic_runs = np.array([130, 220, 290, 145, 190, 270]) # (kts)
Hic_runs = np.array([6000, 6000, 6000, 15000, 15000, 15000]) # (ft)

_____
# *Common Part*
from $\frac{\Delta p_s}{p_s}$ from multiple runs -> $\Delta H_{pc}$ and $\Delta V_{pc}$
_____

## Verification against FAR 25 limits

The civil regs define acceptable errors in terms of $\Delta H_{pc}$ and $\Delta V_{pc}$.

The sequence to obtain $\Delta H_{pc}$ from $\frac{\Delta p_s}{p_s}$ is:

<span style="color: green;">truth source:</span>

<span style="color: green;"> * set reference conditions (ISA/sea level in our case - $H_{c_{ref-alt}}=0$ ft)</span>

<span style="color: green;"> * calculate ambient pressure at ref cond. $p_{a_{ref-alt}}$</span>

<span style="color: red;">ship side:</span>

<span style="color: red;">* with $\frac{\Delta p_s}{p_s}$ and $p_{a_{ref-alt}}$ find static pressure at reference conditions $p_{s_{ref-alt}}$</span>

<span style="color: red;">* find the indicated altitude at ref conditions $H_{ic_{ref-alt}}$</span>

Altitude position correction will be $\Delta H_{pc_{ref-lat}}=H_{c_{ref-alt}}-H_{ic_{ref-alt}}$
___

Define first a **reference altitude** to reduce flight test data to.

In the FAR 25 case, we will use **sea level** at **ISA** conditions.

* $H_{c_{ref-alt}}=0ft$; 
* $T_{ref-alt}=T_0=288.15K$

In [ ]:
# reference altitude = 0ft = sea level
Hc_ref_alt = 0 # (ft)
T_ref_alt = T0 # (K)

ambient pressure at reference altitude:

$p_{a_{ref-alt}}=p_0*(1+\frac{L}{T_0}*H_{c_{ref-alt}}^{(\frac{-g_0}{RL})})$

Note that we know this is $p_0$ for zero altitde and ISA, but if the reference altitude was different...

In [ ]:
pa_ref_alt = p0 * (1 + L / T0 * (Hc_ref_alt*ft2m))**(-g_zero / ( R * L)) # (Pa)
print(f'ambient pressure at reference altitude: {pa_ref_alt} Pa')

static pressure at reference altitude:

$p_{s_{ref-alt}} = \frac{p_{a_{ref-alt}}}{1-\frac{\Delta p_s}{p_s}}$

In [ ]:
ps_ref_alt = pa_ref_alt /  (1 - delta_ps_over_ps_runs) # (Pa)
print(f'static pressure at reference altitude: {ps_ref_alt} Pa')

$\delta_{ic}$ at reference altitude; note subscript *ic* because it includes position error:

$\delta_{ic_{ref-alt}}=\frac{p_{s_{ref-alt}}}{p_0}$

In [ ]:
deltaISA_ic_ref_alt = ps_ref_alt / p0
print(f'deltaISA at reference altitude: {deltaISA_ic_ref_alt}')

Indicated, instrument corrected altitude at reference altitude:

$H_{ic_{ref-alt}}=\frac{T_0}{L}(\delta_{ic_{ref-alt}}^{-(\frac{RL}{g_0})}-1)$

In [ ]:
Hic_ref_alt = T0 / L * ((deltaISA_ic_ref_alt)**(-(R * L / g_zero)) -1) * m2ft # (ft)
print(f'indicated altitude at reference altitude: {Hic_ref_alt} ft')

Altitude position correction:

$\Delta H_{pc_{ref-alt}}=H_{c_{ref-alt}}-H_{ic_{ref-alt}}$

In [ ]:
delta_Hpc_ref_alt = Hc_ref_alt - Hic_ref_alt # (ft)
print(f'delta Hc = {delta_Hpc_ref_alt} ft, with reference altitude={Hc_ref_alt} ft')

## Airpeed (*and Mach*) position corrections

Because we will need $\frac{q_{c}}{p_a}$ to calculate the airspeed correction, we find first the Mach position correction:

To obtain $\Delta M_{pc}$ from $V_{ic_{test}}$ and $\frac{\Delta p_s}{p_s}$:

<span style="color: red;">ship side:</span>

<span style="color: red;">* from our test condition $V_{ic_{test}}$, calculate the differential pressure ratio $(\frac{q_{cic}}{p_0})_{test}$</span>

<span style="color: red;">* from our test condition $H_{ic_{test}}$, calculate $\delta_{ic_{test}}$</span>

<span style="color: red;">* from $(\frac{q_{cic}}{p_0})_{test}$ and $\delta_{ic_{test}}$ get $\frac{q_{cic}}{p_s}$ at test condition</span>

<span style="color: red;">* calculate instrument corrected indicated Mach for test conditions $M_{ic_{test}}$</span>

<span style="color: green;">truth source:</span>

<span style="color: green;">* from $\frac{q_{cic}}{p_s}$ and $\frac{\Delta p_s}{p_s}$, get $\frac{q_{c}}{p_a}$</span>

<span style="color: green;">* from $\frac{q_{c}}{p_a}$ calculate truth Mach</span>

Mach position correction will be $\Delta M_{pc}=M-M_{ic}$
____

To find $\Delta V_{pc}$:

<span style="color: green;">truth source:</span>

<span style="color: green;">* from $\delta_{ref-alt}$ and $\frac{q_c}{p_a}$ find $(\frac{q_c}{p_0})_{ref-alt}$</span>

<span style="color: green;">* calculate the instrument corrected (truth) airspeed at reference conditions $V_{c_{ref-alt}}$</span>

<span style="color: red;">ship side:</span>

<span style="color: red;">* from $(\frac{q_c}{p_0})_{ref-alt}$ and $\frac{\Delta p_s}{p_s}$, get $(\frac{q_{cic}}{p_0})_{ref-alt}$</span>

<span style="color: red;">* calculate the instrument corrected indicated airspeed at reference conditions $V_{ic_{ref-alt}}$</span>

Airspeed position correction will be $\Delta V_{pc_{ref-lat}}=V_{c_{ref-alt}}-V_{ic_{ref-alt}}$
___

Starting from first step:

$(\frac{q_{cic}}{p_0})_{test}=(1+0.2 (\frac{V_{ic_{test}}}{a_0})^2)^\frac{7}{2}-1$

In [ ]:
qcic_over_p0_runs = (1 + 0.2 * ((Vic_runs * kt2ms) / a0)**2)**(7 / 2) -1
print(f'qcic_over_p0 for test runs: {qcic_over_p0_runs}')

$\delta_{ic_{test}}=(1+\frac{L}{T_0}(H_{ic_{test}})^{(\frac{-g_0}{RL})})$

In [ ]:
deltaISA_ic_runs = (1 + L / T0 * (Hic_runs * ft2m))**(-g_zero / (R * L))
print(f'deltaISA_ic for test runs: {deltaISA_ic_runs}')

$\frac{q_{cic}}{p_s}=\frac{q_{cic}}{p_0}*\frac{p_0}{p_s}=(\frac{q_{cic}}{p_0})_{test} * \frac{1}{\delta_{ic_{test}}}$

In [ ]:
qcic_over_ps = qcic_over_p0_runs / deltaISA_ic_runs
print(f'qcic_over_ps for test runs: {qcic_over_ps}')

$M_{ic_{test}}=\sqrt{5*((\frac{q_{cic}}{p_s}+1)^{\frac{2}{7}}-1)}$

And because we will use this expression 2 times, let´s create a function!

In [ ]:
def M_from_q_over_p(q_over_p: float) -> float:
    '''
    Calcucate Mach from q over ps
    '''
    return np.sqrt(5 * ((q_over_p + 1)**(2 / 7) -1))

Mic_runs = M_from_q_over_p(qcic_over_ps)
print(f'indicated Mach for test run: {Mic_runs} ')

$\frac{q_{c}}{p_a}=\frac{\frac{q_cic}{p_s}+1}{1-\frac{\Delta p_s}{p_s}}-1$

In [ ]:
qc_over_pa = (qcic_over_ps + 1) / (1 - delta_ps_over_ps_runs) - 1
print(f'qc_over_pa for test runs: {qc_over_pa}')

$M=\sqrt{5*((\frac{q_{c}}{p_a}+1)^{\frac{2}{7}}-1)}$

In [ ]:
M = M_from_q_over_p(qc_over_pa)
print(f'Truth Mach for test runs: {M}')

Find the delta M

$\Delta M_{pc}=M-M_{ic}$

In [ ]:
delta_Mpc = M - Mic_runs
print(f'Mach position correctins for test runs: {delta_Mpc}')

$\delta_{ref-alt}=(1+\frac{L}{T_0}(H_{c_{ref-alt}})^{(\frac{-g_0}{RL})})$

*note: since we already calculated ambient pressure at reference altitude, we could just do $\frac{p_{a_{ref-alt}}}{p_0}$ ...*

In [ ]:
deltaISA_ref_alt = (1 + L / T0 * (Hc_ref_alt * ft2m))**(-g_zero / (R * L))
print(f'deltaISA for reference altitude: {deltaISA_ref_alt}')

$(\frac{q_c}{p_0})_{ref-alt}=\frac{q_c}{p_a}\delta_{ref-alt}$

In [ ]:
qc_over_p0_ref_alt = qc_over_pa * deltaISA_ref_alt
print(f'qc_over_p0 for test runs at reference altitude: {qc_over_p0_ref_alt}')

$V_{c_{ref-alt}}=a_0*\sqrt{5[((\frac{q_c}{p_0})_{ref-alt}+1)^{\frac{2}{7}}-1]}$

Same idea, let´s create a function.

In [ ]:
def V_from_q_over_p(q_over_p:float) -> float:
    '''
    Calculate airspeed from q_over_ps
    returns airspeed in kts
    '''
    return a0 * np.sqrt(5 * ((q_over_p + 1)**(2 / 7) -1)) * ms2kt # (kts)
Vc_ref_alt = V_from_q_over_p(qc_over_p0_ref_alt) # (kts)
print(f'Instrument corrected, calibrated airspeed for test runs at reference altitude: {Vc_ref_alt} kts')

$(\frac{q_{cic}}{p_0})_{ref-alt}=(\frac{q_c}{p_0})_{ref-alt}-\frac{\Delta p_s}{p_s} \delta_{ic_{ref-alt}}$

In [ ]:
qcic_over_p0_ref_alt = qc_over_p0_ref_alt - delta_ps_over_ps_runs * deltaISA_ic_ref_alt
print(f'qcic_over_p0 for test runs at reference altitude: {qcic_over_p0_ref_alt}')

$V_{ic_{ref-alt}}=a_0*\sqrt{5[((\frac{q_{cic}}{p_0})_{ref-alt}+1)^{\frac{2}{7}}-1]}$

In [ ]:
Vic_ref_alt = V_from_q_over_p(qcic_over_p0_ref_alt) # (kts)
print(f'Instrument correcte, indicated airspeed for test runs at reference altitud: {Vic_ref_alt} kts')

$\Delta V_{pc_{ref-alt}}=V_{c_{ref-alt}}-V_{ic_{ref-alt}}$

In [ ]:
delta_Vpc_ref_alt =Vc_ref_alt - Vic_ref_alt # (kts)
print(f'Airspeed posititon corrections for test runs at reference altitude: {delta_Vpc_ref_alt} kts')

# FAR 25.1323 Compliance Check

## Altitude - 25.1325
(e) Each system must be designed and installed so that the error in indicated pressure altitude, at sea level, with a standard atmosphere, excluding instrument calibration error, does not result in an error of more than ±30 feet per 100 knots speed for the appropriate configuration in the speed range between 1.23 VSR0 with flaps extended and 1.7 VSR1 with flaps retracted. However, the error need not be less than ±30 feet. 

In [ ]:
# FAR 25 limits definitions

k_point = 100 # (kts)
x_limit1 = np.linspace(0, k_point, 10) # (kts)
top_limity1 = np.ones(x_limit1.shape[0]) * 30 # (ft)
bot_limity1 = - top_limity1 # (ft)

x_limit2 = np.linspace(k_point, Vmo, 10) # (kts)
top_limity2 = x_limit2 * 0.3 # (ft)
bot_limity2 = - top_limity2 # (ft)

In [ ]:
# set graph size
plt.rcParams['figure.figsize'] = [12, 7]

# plot limits
plt.plot(x_limit1, top_limity1, 'r', label='FAR 25 limit')
plt.plot(x_limit1, bot_limity1, 'r')
plt.plot(x_limit2, top_limity2, 'r')
plt.plot(x_limit2, bot_limity2, 'r')

# plot data
plt.plot(Vic_runs, delta_Hpc_ref_alt, '+b', label='Flight Test', markersize=10)

# label, grid, etc
plt.xlabel( 'Vic (kts)')
plt.ylabel('delta Hpc (ft)')
plt.title('Altitude Error Plot')
plt.grid(True)
plt.legend()
plt.show()

## Airspeed - 25.1323

25.1323 (c) states:
The airspeed error of the installation, excluding the airspeed indicator instrument calibration error, may not exceed three percent or five knots, whichever is greater, throughout the speed range, from - 

(1) VMO  to 1.23 VSR1, with flaps retracted; and 

(2) 1.23 VSR0 to VFE with flaps in the landing position. 

In [ ]:
# FAR 25 limits definitions
k_point = 5 / 0.03 # (kts)
x_limit1 = np.linspace(Vsr0, k_point, 10) # (kts)
top_limity1 = np.ones(x_limit1.shape[0]) * 5 # (kts)
bot_limity1 = - top_limity1 # (kts)

x_limit2 = np.linspace(k_point, Vmo, 10) # (kts)
top_limity2 = x_limit2 * 0.03 # (kts)
bot_limity2 = - top_limity2 # (kts)

In [ ]:
# plot limits

# plot limits
plt.plot(x_limit1, top_limity1, 'r', label='FAR 25 limit')
plt.plot(x_limit1, bot_limity1, 'r')
plt.plot(x_limit2, top_limity2, 'r')
plt.plot(x_limit2, bot_limity2, 'r')

# plot data
plt.plot(Vic_runs, delta_Vpc_ref_alt, '+b', label='Flight Test', markersize=10)

# label, grid, etc
plt.xlabel( 'Vic (kts)')
plt.ylabel('delta Vpc (kts)')
plt.title('Speed Error Plot')
plt.grid(True)
plt.legend()
plt.show()